In [2]:
import glob
import pandas as pd


input_Loc ='E:\_Projects\DEP POR\Key Account\Input\\'
output_Loc = 'E:\_Projects\DEP POR\Key Account\Output\\'

files = glob.glob(input_Loc+'IndHierData_??????????.csv')

#print(files)

# 0 - KA pull from SQL | 1 - KA get from local
SQL = 0


consolidated_file = pd.DataFrame()


for x in files:
    print(x)
    df = pd.read_csv(x,low_memory=False)
    consolidated_file = pd.concat([consolidated_file,df],axis=0)




consolidated_file = consolidated_file[['REGION','AREA','COUNTRY','COUNTRY_CODE','ZONE','DESCRIPTION','KEY_ACCOUNT','MASTER_CUSTOMER','MASTER_CUSTOMER_NUMBER']]


consolidated_file.drop_duplicates(inplace=True)


    
consolidated_file.to_csv(output_Loc+'Consolidate.csv',index=False)




E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912919930.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912919963.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912920059.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912920135.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912920165.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912920288.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912921258.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912921736.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912921789.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912921882.csv
E:\_Projects\DEP POR\Key Account\Input\IndHierData_1912922112.csv


In [2]:
Key_Master = pd.read_excel(input_Loc+'7.3_Country-Key_Account_Heirarchy.xlsb',engine='pyxlsb')


Key_Master.drop_duplicates(subset=['Master Customer Number'],inplace=True)


Master = pd.merge(Key_Master, consolidated_file, right_on='MASTER_CUSTOMER_NUMBER', left_on='Master Customer Number', how='outer')


#Master.to_csv(output_Loc+'Master.csv',index=False)


# Check if "Master Customer Name" column is null (blank) and "MASTER_CUSTOMER_NUMBER" column is not null
mask = Master['Master Customer Number'].isnull() & Master['MASTER_CUSTOMER_NUMBER'].notnull()
Master.loc[mask, 'New_MCN'] = "New MCN Found"
# Assign new values if both conditions are met
Master.loc[mask, 'Master Customer Number'] = Master["MASTER_CUSTOMER_NUMBER"]
Master.loc[mask, 'Region'] = Master["REGION"]
Master.loc[mask, 'Area'] = Master["AREA"]
Master.loc[mask, 'Country Name'] = Master["COUNTRY"]
Master.loc[mask, 'Country Code'] = Master["COUNTRY_CODE"]
Master.loc[mask, 'Zone'] = Master["ZONE"]
Master.loc[mask, 'Key Account'] = Master["KEY_ACCOUNT"]
Master.loc[mask, 'Master Customer Name'] = Master["MASTER_CUSTOMER"]
Master.loc[mask, 'Key Account Descr'] = Master["DESCRIPTION"]

Master.loc[Master['KEY_ACCOUNT'].notnull(),'Key Account'] = Master['KEY_ACCOUNT']

Master = Master.iloc[:,0:10]

Master.drop_duplicates(inplace=True)



In [3]:
print('#######################################################\n')
print('     Pulling KA file from SQL.....................!')
print('\n#######################################################\n')



#######################################################

     Pulling KA file from SQL.....................!

#######################################################



In [5]:
import oracledb
import pandas as pd
import numpy as np

if SQL == 0:

    # Enable thick mode
    oracledb.init_oracle_client(lib_dir=r"E:\softwares\instantclient-basic-windows.x64-23.5.0.24.07\instantclient_23_5")

    # Connection details
    dsn_tns = oracledb.makedsn('153.84.75.216', 1521, service_name='ebs_ATMPROD')


    # Use a context manager to handle the connection
    with oracledb.connect(user='erp_user', password='erp_pass', dsn=dsn_tns) as conn:
        data_sod_bom = pd.DataFrame()

        query = f"""
        select * from
NCR_DEM_KEY_ACCOUNT_ASSG
        """

        # Fetch data into a pandas DataFrame and append to the main DataFrame
        SQL_Master = pd.read_sql(query, con=conn)
        
    # Save the combined DataFrame to a CSV file
    SQL_Master.to_csv('Output/ref/SQL.csv', index=False)
else:
    SQL_Master = pd.read_csv('Output/ref/SQL.csv')


DatabaseError: ORA-12170: Cannot connect. - timeout of -s for host 153.84.75.217 port 1521. (CONNECTION_ID=DBDaTz+RTGus1d56jffZ+A==)
Help: https://docs.oracle.com/error-help/db/ora-12170/

In [5]:

################## KEY Account Hiearachi

#SQL_Master = pd.read_excel(input_Loc + 'SQL.xlsx')


SQL_Master = SQL_Master[['MASTER_CUST_NUM','KEY_ACCOUNT','MASTER_CUST_NAME']]

SQL_Master.drop_duplicates(subset=['MASTER_CUST_NUM'],inplace=True)

Master = pd.merge(Master, SQL_Master, left_on='Master Customer Number',right_on='MASTER_CUST_NUM',how='outer')


# Check if "Master Customer Name" column is null (blank) and "MASTER_CUSTOMER_NUMBER" column is not null
mask = Master['Master Customer Number'].isnull() & Master['MASTER_CUST_NUM'].notnull()

# Assign new values if both conditions are met
Master.loc[mask, 'Master Customer Number'] = Master["MASTER_CUST_NUM"]
Master.loc[mask, 'Key Account'] = Master["KEY_ACCOUNT"]
Master.loc[mask, 'Master Customer Name'] = Master["MASTER_CUST_NAME"]

Master.loc[Master['Key Account'] != Master['KEY_ACCOUNT'],'SQL_Key_Account'] = Master['KEY_ACCOUNT']



Master = Master.filter(regex='^(?!.*KEY_ACCOUNT$)')
Master = Master.filter(regex='^(?!.*MASTER_CUST_NAME$)')
Master = Master.filter(regex='^(?!.*MASTER_CUST_NUM$)')


Master.drop_duplicates(inplace=True)
Master.to_csv('Output/Master_KeyAccount.csv', index=False)


In [6]:
print('#######################################################\n')
print('                     Automation Completed')
print('\n#######################################################\n')

#######################################################

                     Automation Completed

#######################################################

